<h2>Информация о маркетинговых кампаниях банка<h2>
<h4>Цель — предсказать, подпишется ли клиент на срочный депозит<h4>

In [ ]:
!pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 1.0/124.9 MB 4.6 MB/s eta 0:00:28
    --------------------------------------- 2.1/124.9 MB 4.9 MB/s eta 0:00:26
   - -------------------------------------- 4.7/124.9 MB 7.7 MB/s eta 0:00:16
   -- ------------------------------------- 7.6/124.9 MB 9.6 MB/s eta 0:00:13
   --- ------------------------------------ 10.0/124.9 MB 9.9 MB/s eta 0:00:12
   --- ------------------------------------ 12.1/124.9 MB 9.8 MB/s eta 0:00:12
   ---- ----------------------------------- 14.2/124.9 MB 10.0 MB/s eta 0:00:12
   ----- ---------------------------------- 16.8/124.9 MB 10.2 MB/s eta 0:00:11
   ------ --------------------------------- 19.1/124.9 MB 10.4 MB/s eta 0:00:11
   ------ --------------------------------- 21.5/124.9 MB 10.5 MB/s eta 0:00:10
   ------- -------------------------------- 23.9/124.9 MB 10.6 MB/s 


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
'''
-импортируем pandas и matplotlib для анализа
и визуализации данных

-импортриуем sklearn для создания модели, которая
будет решать поставленную задачу

-numpy для числовых вычеслений

'''

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix,make_scorer,average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from xgboost import XGBClassifier



import numpy as np

In [34]:
'''

загружаю данные и проверяю как они считались

'''

data = pd.read_csv(r"/content/bank-full.csv", sep=';')#здесь важно указать разделитель т.к по умолчанию
                                                                            #используется ',' а здесь у нас ';'


data.head()


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [35]:
data.info()
#пропущенных значений нет и это хорошо

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [36]:
'''
-подготавливаем данные для работы
переводим все значения в числовые типы данных

'''

def categorical_to_numeric(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """
    Converts a categorical column in a Pandas DataFrame to numerical using One-Hot Encoding.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column (str): The name of the categorical column to convert.

    Returns:
        pd.DataFrame: The DataFrame with the specified column converted to numerical using One-Hot Encoding.
    """
    # Проверяем, является ли тип данных столбца категориальным
    if pd.api.types.is_object_dtype(df[column]): # Use pandas function for checking object dtype
        # Создаем экземпляр OneHotEncoder
        onehot_encoder = OneHotEncoder(drop='first', sparse_output=False)  # drop='first' для избежания проблемы мультиколлинеарности, sparse_output=False to return a numpy array not a sparse matrix

        # Применяем OneHotEncoder и получаем массив
        encoded_array = onehot_encoder.fit_transform(df[[column]])

        # Создаем DataFrame из закодированных данных
        encoded_df = pd.DataFrame(encoded_array, columns=onehot_encoder.get_feature_names_out([column]))

        # Объединяем исходный DataFrame с закодированными данными
        df = df.drop(column, axis=1)  # Удаляем исходный столбец
        df = pd.concat([df, encoded_df], axis=1)  # Добавляем закодированные столбцы

        df = df.reset_index(drop=True)  # Reset index after concat to avoid issues if the index is not consecutive
        encoded_df = encoded_df.reset_index(drop=True)  # Reset index after concat to avoid issues if the index is not consecutive
    return df


In [37]:
for column in data.columns:
    if np.issubdtype(data[column], np.object_):
        data = categorical_to_numeric(data,column)

In [38]:
df = data[data['y_yes'] == 0].sample(frac=1.0, random_state=42)
df = df.iloc[0:5289]
new_data = pd.concat([data[data['y_yes'] == 1], df])

In [39]:
X = new_data.drop('y_yes', axis=1)  # Используем исходный столбец 'y'
y = new_data.y_yes

# Стратифицированное разделение для сохранения исходного распределения классов
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [40]:
class_weight = {
    0.0 : 1,
    1.0 : 5
}

In [42]:
clf = RandomForestClassifier(random_state=42)



# Задаем распределения параметров (вместо списков значений)
params = {
    'n_estimators' : [100, 200, 300],
    'criterion': ['gini', 'entropy'],
    'max_depth' : range(1, 10)
}

grid_search = GridSearchCV(clf, param_grid=params, cv=5, scoring='accuracy', n_jobs=-1)


grid_search.fit(X_train, y_train)

best_tree = grid_search.best_estimator_
pred_y = best_tree.predict(X_test)



In [43]:
print("Best params:", grid_search.best_params_)
print("CV F1:", grid_search.best_score_)

# Полный отчет
print(classification_report(y_test, pred_y))
print("AUC-PR:", average_precision_score(y_test, best_tree.predict_proba(X_test)[:,1]))

Best params: {'criterion': 'gini', 'max_depth': 9, 'n_estimators': 300}
CV F1: 0.8398737535590157
              precision    recall  f1-score   support

         0.0       0.87      0.80      0.84      1058
         1.0       0.82      0.88      0.85      1058

    accuracy                           0.84      2116
   macro avg       0.85      0.84      0.84      2116
weighted avg       0.85      0.84      0.84      2116

AUC-PR: 0.9063476476197732


In [50]:
importances = best_tree.feature_importances_
feature_names = X_train.columns
pd.Series(importances, index=feature_names).sort_values(ascending=False)


,0
duration,0.443532
poutcome_success,0.072624
contact_unknown,0.064489
housing_yes,0.043112
pdays,0.042604
age,0.041489
balance,0.039281
previous,0.029729
day,0.025759
poutcome_unknown,0.023910


In [53]:
threshold = 0.02 # Процент от максимальной важности (измените по необходимости)

from sklearn.feature_selection import SelectFromModel
sfm = SelectFromModel(best_tree, threshold=threshold)
sfm.fit(X_train, y_train)

selected_features_sfm = X_train.columns[sfm.get_support()]
X_train_sfm = sfm.transform(X_train)
X_test_sfm = sfm.transform(X_test)

print("Выбранные признаки с SelectFromModel:", selected_features_sfm.tolist())


Выбранные признаки с SelectFromModel: ['age', 'balance', 'day', 'duration', 'pdays', 'previous', 'housing_yes', 'contact_unknown', 'poutcome_success', 'poutcome_unknown']


In [55]:
clf_1 = RandomForestClassifier(random_state=42,
                             criterion='gini',
                             max_depth=9,
                             n_estimators=500
                            )
clf_1.fit(X_train_sfm, y_train)
pred = clf_1.predict(X_test_sfm)

print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         0.0       0.85      0.79      0.82      1058
         1.0       0.81      0.86      0.83      1058

    accuracy                           0.83      2116
   macro avg       0.83      0.83      0.83      2116
weighted avg       0.83      0.83      0.83      2116



In [56]:
from sklearn.linear_model import LogisticRegression

model_selected_sfm = LogisticRegression(random_state=42)
model_selected_sfm.fit(X_train_sfm, y_train)
y_pred_sfm = model_selected_sfm.predict(X_test_sfm)
print("Отчет о классификации (SelectFromModel):")
print(classification_report(y_test, y_pred_sfm))

Отчет о классификации (SelectFromModel):
              precision    recall  f1-score   support

         0.0       0.78      0.82      0.80      1058
         1.0       0.81      0.77      0.79      1058

    accuracy                           0.79      2116
   macro avg       0.80      0.79      0.79      2116
weighted avg       0.80      0.79      0.79      2116



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
